In [1]:
import argparse
import csv
import json
import os
import pathlib
import re
import utils

from collections import defaultdict

In [50]:
generalization = utils.read_jsonl("../data/experiments/single_stimuli_dative_simulation/generalization.jsonl")
adaptation = utils.read_jsonl("../data/experiments/single_stimuli_dative_simulation/adaptation.jsonl")

In [52]:
adapt2acro = {}
for entry in adaptation:
    adapt2acro[entry['hypothesis_id']] = utils.generate_acronym(entry) 

In [53]:
adapt2acro

{1: 'paspas',
 2: 'paspis',
 3: 'pasnal',
 4: 'pasnas',
 5: 'pasnil',
 6: 'pasnis',
 7: 'pispas',
 8: 'pisnal',
 9: 'pisnas',
 10: 'pisnil',
 11: 'pisnis',
 12: 'nalpas',
 13: 'nalpis',
 14: 'nalnal',
 15: 'nalnas',
 16: 'nalnil',
 17: 'nalnis',
 18: 'naspas',
 19: 'naspis',
 20: 'nasnal',
 21: 'nasnas',
 22: 'nasnil',
 23: 'nasnis',
 24: 'nilpas',
 25: 'nilpis',
 26: 'nilnal',
 27: 'nilnas',
 28: 'nilnil',
 29: 'nilnis',
 30: 'nispas',
 31: 'nispis',
 32: 'nisnal',
 33: 'nisnas',
 34: 'nisnil',
 35: 'nisnis'}

In [11]:
len(generalization)

1368

In [35]:
utils.generate_acronym(generalization[0])

'paspas'

In [76]:
def metadata(dir):
    exp_dir = dir.split("/")[-1]
    item_id, hyp_id, hyp_instance, lr, adaptation_dative, _ = exp_dir.split('_')
    training_summary = utils.read_json(f"{dir}/training_summary.json")
    return {
        'item_id': int(item_id),
        'hypothesis_id': int(hyp_id),
        'hypothesis_instance': int(hyp_instance),
        'lr': float(lr),
        'adaptation_dative': adaptation_dative,
        'val_performance': training_summary['best_val_performance'],
        'best_epoch': training_summary['best_epoch'],
        'train_loss': training_summary['train_loss']
    }

In [81]:
tuple(metadata("../data/results/single_stimuli_dative_simulation/smolm-autoreg-bpe-seed_111/raw/1_1_1_0.1_do_results").values()) + (1,2,3)

(1, 1, 1, 0.1, 'do', 2.635527736345926, 21, 2.0407309532165527, 1, 2, 3)

In [82]:
def compile_generalization_results(dir, genset=generalization, forcsv=False):
    exp_metadata = metadata(dir)
    # print(exp_metadata)
    # exp_metadata['adaptation_dative']
    adaptation_feature_config = adapt2acro[exp_metadata['hypothesis_id']]

    generalization_results = utils.read_csv_dict(f"{dir}/generalization_results.csv")

    compiled_results = {
        'initial': {'do': defaultdict(list), 'pp': defaultdict(list)},
        'final': {'do': defaultdict(list), 'pp': defaultdict(list)},
        'best': {'do': defaultdict(list), 'pp': defaultdict(list)}
    }

    # return generalization_results
    for gen, parse in zip(genset * 3, generalization_results):
        acronym = utils.generate_acronym(gen)
        compiled_results[parse['model_state']][gen['dative']][acronym].append(float(parse['score']))

    # get means
    

    if forcsv:
        means = []
        for state, datives in compiled_results.items():
            for dative, hyps in datives.items():
                for hyp, scores in hyps.items():
                    mean = sum(scores)/len(scores)
                    means.append(tuple(exp_metadata.values()) + (adaptation_feature_config, state, dative, hyp, mean))
    else:
        means = {
            **exp_metadata,
            'adaptation_feature_config': adaptation_feature_config,
            'initial': {'do': {}, 'pp': {}},
            'final': {'do': {}, 'pp': {}},
            'best': {'do': {}, 'pp': {}}
        }
        for state, datives in compiled_results.items():
            for dative, hyps in datives.items():
                for hyp, scores in hyps.items():
                    means[state][dative][hyp] = sum(scores) / len(scores)

    return means


In [134]:
parsed = compile_generalization_results("../data/results/single_stimuli_dative_simulation/smolm-autoreg-bpe-seed_111/raw/1_1_1_0.1_do_results")

In [141]:
def flatten_results(result):
    # item_id, hypothesis_id, hypothesis_instance, lr, adaptation_dative, val_performance, best_epoch, train_loss, adaptation_feature_config, state, dative, hypothesis, mean
    flattened = []
    for state in ['initial', 'final', 'best']:
        for dative, hyps in result[state].items():
            for hyp, mean in hyps.items():
                flattened.append((result['item_id'], result['hypothesis_id'], result['hypothesis_instance'], result['lr'], result['adaptation_dative'], result['val_performance'], result['best_epoch'], result['train_loss'], result['adaptation_feature_config'], state, dative, hyp, mean))

    return flattened

In [142]:
len(flatten_results(parsed))

150

In [143]:
results = defaultdict(list)

i = 0
j = 0
for dir in pathlib.Path("../data/results/single_stimuli_dative_simulation/smolm-autoreg-bpe-seed_111/raw").glob("*"):
    if dir.is_dir():
        try:
            parsed = compile_generalization_results(str(dir))
            results[(parsed['item_id'], parsed['hypothesis_id'], parsed['hypothesis_instance'])].append((parsed['lr'], parsed['val_performance'], parsed))
            # parsed = compile_generalization_results(str(dir), forcsv=True)
            # results[(parsed[0][0], parsed[0][1], parsed[0][2])].append((parsed[0][3], parsed[0][5], parsed))
        except:
            j += 1

results = dict(results)

In [155]:
lr, val, gen_res = max(results[6,1,1], key=lambda x: x[1])

# len(flatten_results(gen_res))

# gen_res['initial']['pp']

flatten_results(gen_res)

[(6,
  1,
  1,
  0.1,
  'pp',
  3.866237827936809,
  47,
  1.807220220565796,
  'paspas',
  'initial',
  'do',
  'nalpas',
  -7.696172037789988),
 (6,
  1,
  1,
  0.1,
  'pp',
  3.866237827936809,
  47,
  1.807220220565796,
  'paspas',
  'initial',
  'do',
  'nalpis',
  -7.518865053043809),
 (6,
  1,
  1,
  0.1,
  'pp',
  3.866237827936809,
  47,
  1.807220220565796,
  'paspas',
  'initial',
  'do',
  'nalnal',
  -6.768294700356417),
 (6,
  1,
  1,
  0.1,
  'pp',
  3.866237827936809,
  47,
  1.807220220565796,
  'paspas',
  'initial',
  'do',
  'nalnas',
  -7.687125583027685),
 (6,
  1,
  1,
  0.1,
  'pp',
  3.866237827936809,
  47,
  1.807220220565796,
  'paspas',
  'initial',
  'do',
  'naspas',
  -8.31981699965721),
 (6,
  1,
  1,
  0.1,
  'pp',
  3.866237827936809,
  47,
  1.807220220565796,
  'paspas',
  'initial',
  'do',
  'naspis',
  -8.06561190582985),
 (6,
  1,
  1,
  0.1,
  'pp',
  3.866237827936809,
  47,
  1.807220220565796,
  'paspas',
  'initial',
  'do',
  'nasnal',
  -

In [145]:
max(results[(2,1,1)], key=lambda x: x[1])

KeyError: (2, 1, 1)

In [107]:
i

2100

In [102]:
i, j

(2100, 2070)

In [99]:
350 * 6 + 175 * 6

3150